In [1]:
def writeVTKPoints(outFileName,x,y,z, data): 
    pointsNumber=x.size 
    m=open(outFileName,'w')
    m.write('# vtk DataFile Version 2.0\n') 
    m.write('hypocenters\n') 
    m.write('ASCII\n')
    m.write('DATASET POLYDATA\n')
    m.write('POINTS '+str(pointsNumber)+' float\n') 
    for node in np.arange(pointsNumber):
        m.write(str(x[node])+' '+str(y[node])+' '+str(z[node])+' \n')
    m.write('POINT_DATA '+str(pointsNumber)+'\n')
    m.write('SCALARS scalars float 1\n')
    m.write('LOOKUP_TABLE default\n')
    for node in np.arange(pointsNumber):
        m.write(str(data[node])+' \n')
    #m.write(’POLYGONS ’+str(trianglesNumber)+’ ’+str(trianglesNumber*4)+’\n’) for triangle in np.arange(trianglesNumber):
    #m.write(’3 ’+str(triangles[triangle,0])+’ ’+str(triangles[triangle,1])+’ ’+str(triangles[triangle,2])+’ \n’)
    m.close() 
    return open(outFileName, 'r')

# Class for easy access to data

In [2]:
### class for easy access to INGV data. 
class hypoData:
    
    def __init__(self, minMag=None, selectSequence = None):
            self.variable_folder=variable_folder
            self.sort=False
            self.minMag=minMag
            self.selectSequence = selectSequence
            self.getHypocenters()
            self.getFocalAvailable()
            
    def getHypocenters(self, selectSequence=None):
        if selectSequence is not None:
            self.selectSequence = selectSequence
        
        if self.selectSequence is None:
            self.retINGV = selectedINGV(minMag=self.minMag, path=variable_folder)
       
        elif self.selectSequence == '201617':
            self.retINGV = loadSecondSequence(minMag=self.minMag)
        
        
        if self.sort==True:
            inds = np.flip(self.retINGV[3].argsort(), 0)
            self.retINGV = [arr[inds] for arr in self.retINGV]
        (self.lat, self.lon, self.z, self.mL, self.rad, self.st1, self.dp1, self.rk1,
            self.st2, self.dp2, self.rk2, self.time, self.slip) = self.retINGV
        
        self.geogToMet()
        self.p = np.array( [self.x, self.y, self.z] ).T
        
    def getFocalAvailable(self):
        self.FA = ~ (
            (self.st1==0) * (self.st2==0) *
            (self.dp1==0) * (self.dp2==0) * 
            (self.rk1==0) * (self.rk2==0)
            )
        return self.FA
        
    def geogCenter(self, lonCenter = None, latCenter = None):
        if lonCenter is None or latCenter is None:
            self.lonCent = self.lon[self.mL==np.amax(self.mL)]
            self.latCent = self.lat[self.mL==np.amax(self.mL)]
        else:
            self.lonCent = lonCenter
            self.latCent = latCenter
            
    def geogToMet(self):
        try:
            self.lonCent, self.latCent
        except:
            self.geogCenter()
        self.y, self.x = ChangeAxis(self.latCent, self.lonCent, self.lat, self.lon)
        
    def metToGeog(self, xExternal, yExternal):
        """returns (lat, lon)
        
        For points in xyz coordinates that exist around the coordinate system in this object,
            this function returns them to lat/lon. """
        lat, lon = xyToLonLat(self.latCent, self.lonCent, xExternal, yExternal)
        
        return lat, lon
    
    def getDists(self):
        self.distance = distances(self.x, self.y, self.z, 
                  self.x, self.y, self.z)
        
    def plot2d(self):
        fig = plt.figure(figsize=(10,10))
        plt.scatter(self.x, self.y, c=self.labels, edgecolors='k', cmap='tab20')
        plt.colorbar()
        plt.show()
        
    def plot3d(self):
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(self.x, self.y, self.z, c=self.labels)
        ax.view_init(10, 10)
        plt.show()
            
        
    def vtkSave(self, fName = None):
        if fName==None:
            fName = self.fName
        writeVTKPoints(variable_folder+fName, self.x, self.y, self.z, 
                      self.labels)
        
    def saveAllVTK(self):
        writeVTKPoints(variable_folder+'AllVTK.vtk', self.x, self.y, self.z, 
                      np.zeros(self.x.size) )
        
    def plot3dclusters(self, pointSize=4, lineWidthR = 1/10, save = False, name = 'to_delete', 
                      x=None, y=None, z=None, labels=None):
        if x is None:
            x = self.x
            y = self.y
            z = self.z
            labels = self.labels
        
        scatterPlot = [go.Scatter3d(
            x=x,
            y=y,
            z=z,
            mode='markers',
            marker=dict(
                size=pointSize,
                color=labels,
                colorscale='Jet',
                line=dict(
                        color='rgb(0,0,0)',
                        width=lineWidthR 
                    ),
                opacity=1) )]
        
        ############################ 
        textFontDict = {'color':'red', 'size':18}
        ### north arrow
#         xNA = 6.5e3; yNA = 10e3; zNA = -14e3
        xR = 10e3
        xNA = np.amax(xR-3e3)
        yNA = 28e3#np.amax(self.y)
        zNA = -14e3#np.amin(self.z)
        yBase = yNA - 5e3
        northLine = go.Scatter3d(
            x=[xNA, xNA], y=[yBase, yNA], z=[zNA, zNA],
            text = np.array(['', 'N']),
            mode='lines+text',#Change?
            line  =dict(width = 5, color = '#7f7f7f'),
            textposition = 'middle left',
            textfont = textFontDict
        )

        northCone = dict(
            type = 'cone',
            colorscale = 'Greys',
            showscale = False,
            x = [xNA], y = [yNA], z = [zNA],
            u = [0], v = [2e3], w = [0]
            )
        
        upLine = go.Scatter3d(
            x=[xNA, xNA], y=[yBase, yBase], z=[zNA, zNA+5e3],
            text = np.array(['', 'UP']),
            mode='lines+text',#Change?
            line  =dict(width = 5, color = '#7f7f7f'),
            textposition = 'middle left',
            textfont = textFontDict
        )
        
        upCone = dict(
            type = 'cone',
            colorscale = 'Greys',
            showscale = False,
            x = [xNA], y = [yBase], z = [zNA+5e3],
            u = [0], v = [0], w = [2e3]
            )
        ###
        
        ###  scale bar      
        xN = np.array([xR, xR, xR])
        yN = np.array([-10e3, -5e3, 0])+yNA
        zN = np.array([zNA, zNA, zNA])
        scaleBar = go.Scatter3d(
            x=xN,
            y=yN,
            z=zN,
            text = np.array(['', '10 km', '']),
            mode='lines+text',
            line  =dict(width = 15, color = '#7f7f7f'),
            textposition='middle right',
            textfont = textFontDict
        )
        ###
        scatterPlot.append(scaleBar)
        scatterPlot.append(northLine)
        scatterPlot.append(northCone)
        scatterPlot.append(upLine)
        scatterPlot.append(upCone)
        #########################
        
        layout = go.Layout(
            showlegend=False,
            autosize=False,
            width=1000,
            height=1000,
            scene=dict(
                camera=dict(eye=dict(x=1.75, y=-0.7, z= 0.75) ),
                aspectmode = 'data',
                xaxis = dict(showticklabels=False, title=''),
                yaxis = dict(showticklabels=False, title=''),
                zaxis = dict(showticklabels=False, title='')
                      )  
            )

        fig2 = go.Figure(data=scatterPlot, layout=layout)
        
        if not save:
            plot(fig2)
        elif save:
            py.iplot(fig2, filename = name)
    

# Clustering classes

As of 09 19 2018, 12:47, I have only included the clustering classes that were in use for the methods. The other classes are still available in the clustering file. 

In [3]:
class spClust(hypoData):
    def __init__(self, minMag=None, selectSequence = None, sigma = None):
        super().__init__(minMag=minMag, selectSequence = selectSequence)
        
        self.fName = 'SpectralClustering.vtk'
        self.sigma = sigma
    
    def cluster(self):
        self.getDists()
        if self.sigma is None:
            self.sigma = sp.distance[~(sp.distance==0)].std()
        
        self.similarity = np.exp(-1/2 * (self.distance / self.sigma)**2)

        self.results = sklearn.cluster.SpectralClustering(
            n_clusters=self.n_clusters,
            n_init=self.n_init,
            assign_labels=self.assign_labels,
            random_state=np.random.RandomState(1),
            affinity = 'precomputed',
            ).fit(self.similarity)
        
        self.labels = self.results.labels_

In [4]:
class dbClust(hypoData):
    def __init__(self, minMag=None, selectSequence = None):
        super().__init__(minMag=minMag, selectSequence = selectSequence)
        self.sort=False
        
    def fitdbscan(self, weight=None):        
#         self.pTrans = sklearn.preprocessing.StandardScaler().fit_transform(self.p)
        self.pTrans=self.p
        
        if weight is None:       
            self.db = sklearn.cluster.DBSCAN(
                eps=self.eps, min_samples=self.min_samples, 
                n_jobs=multiprocessing.cpu_count()-1
                ).fit(self.pTrans)
        elif weight is not None:
            self.db = sklearn.cluster.DBSCAN(
                eps=self.eps, min_samples=self.min_samples, 
                n_jobs=multiprocessing.cpu_count()-1
                ).fit(self.pTrans, weight)   
        
        self.labels=self.db.labels_

In [ ]:
# Very early stages of trying optics clustering, which automatically determines eps and eps can be variable, unlike DBSCAN
class opticsClust(hypoData):
    def __init__(self, minMag=None, selectSequence = None):
        super().__init__(minMag=minMag, selectSequence = selectSequence)
        self.sort=False
        
    def fitdbscan(self, weight=None):        
#         self.pTrans = sklearn.preprocessing.StandardScaler().fit_transform(self.p)
        self.pTrans=self.p
    
        self.db = sklearn.cluster.OPTICS(min_samples=self.min_samples, 
                                         metric = 'euclidean', max_eps = 3e3
            ).fit(self.pTrans) 
        
#         if weight is None:       
#             self.db = sklearn.cluster.OPTICS(
#                 eps=self.eps, min_samples=self.min_samples, 
#                 n_jobs=multiprocessing.cpu_count()-1
#                 ).fit(self.pTrans)
#         elif weight is not None:
#             self.db = sklearn.cluster.OPTICS(
#                 eps=self.eps, min_samples=self.min_samples, 
#                 n_jobs=multiprocessing.cpu_count()-1
#                 ).fit(self.pTrans, weight)   
        
        self.labels=self.db.labels_

# Spline interpolation


In [5]:
def normalToStrikeDip(normals):
    """returns (strikes, dips)
    
    input normals as shape (n, 3)
    
    Assumes dip only varies between 0 and pi/2.
    Follows right hand rule."""
    if normals.shape == np.array([0,0,0]).shape:
        normals = np.array([normals])
        oneDimension = True
    
    points = normals.shape[0]
    strikes = np.zeros(points)
    dips = np.zeros(points)
    
    # To obey the right hand rule, normals must face up???
    # Also makes dip angle less than or equal to 90
    flip = normals[:,2]<0.
    normals[flip] = - normals[flip]

    dips = np.arctan2( np.sqrt(normals[:, 0]**2 + normals[:, 1]**2),  normals[:,2] )
    strikes = np.arctan2(-normals[:,1], normals[:,0])
    strikes[strikes<0]+=2*np.pi
        
    return strikes, dips

In [15]:
def angleBetweenVectors(a, b, max_90 = False):
    """ 
    Options for shapes of a and b:
    
    Both a and b are of shape (n, 3):
        Returns angles between a[i] and b[i]
        
    a is (3) and b is (n,3):
        Returns angle between a and b[i]
        
    a and b are shape (3):
        Return angle between the two vectors
    
    Only has precision to 1e-8
        
    """
    points = int(b.size/3)
    
    if a.size > b.size:
        raise Exception('b cannot be smaller than a')
    
    if a.shape==np.array([0,0,0]).shape:
        a = a * np.ones( (points,3) )
    if b.shape==np.array([0,0,0]).shape:
        b = b * np.ones( (points,3) )
    
    theta = np.zeros(points)
    for i in np.arange(points):
        cosine = np.dot(a[i], b[i]) / (norm(a[i]) * norm(b[i]))
        if cosine > 1:
            theta[i] = np.pi/2
        elif cosine <-1:
            theta[i] =-np.pi/2
        else:
            theta[i] = np.arccos( cosine )
            
    if max_90:
        theta[theta > np.pi/2] = np.pi - theta[theta > np.pi/2]
            
    return theta

# import numpy as np
# from numpy.linalg import norm
# a = np.array([0, 0, 1]) 
# b = np.array([0, 1, -1])
# np.rad2deg(angleBetweenVectors(a, b, max_90 = True))

array([45.])

In [11]:
class optimumPlane:
    def __init__(self, xP, yP, zP, exp = 2, averageCenter=True):
        self.exp = exp
        
        self.xP = xP
        self.yP = yP
        self.zP = zP
        
        #### TODO: Misleading. These are only the beginning center points. 
        self.xC = np.average(self.xP)
        self.yC = np.average(self.yP)
        self.zC = np.average(self.zP)
        
        self.runMinimize()
        
    def getError(self, ins):
        strike, dip, xC, yC, zC = ins
        normal, __ = faultUnitVectors(strike, dip, 0)

        errors = distFromPlane(
                self.xP, self.yP, self.zP,
                xC, yC, zC,
                normal[0], normal[1], normal[2]
                )
        
        errTot = np.sum( np.abs(errors)**self.exp )
        return errTot
    
    def runMinimize(self):
        bounds = ( (0, 2*np.pi), 
                  (0, np.pi/2), 
                  (np.amin(self.xP),np.amax(self.xP)), 
                  (np.amin(self.yP),np.amax(self.yP)), 
                  (np.amin(self.zP),np.amax(self.zP)) )
        startStrike = np.linspace(0, 2*np.pi, 10)
        startStrike = startStrike[1:]
        startStrike = startStrike[:-1]
        startDip = np.linspace(0, np.pi/4, 6)
        startDip = startDip[1:]
        startDip = startDip[:-1]
        
        # Invert through several starting points. 
        results = []
        errors = []
        for i in np.arange(startStrike.size):
            for j in np.arange(startDip.size):
                
                resultTemp = scipy.optimize.minimize(
                    self.getError, np.array([startStrike[i], startDip[j], self.xC, self.yC, self.zC]),
                    method='CG')
                
                results.append( resultTemp )  
                errors.append( resultTemp.fun )
            
        # Find best result
        errors = np.array(errors)
        errSort = np.argsort(errors)
        minErr = errSort==np.amin(errSort)
        minInd=int(np.arange(minErr.size)[minErr])
        self.result = results[minInd]

        #Angles and Position, the inverted info. 
        self.strike, self.dip, self.xC, self.yC, self.zC = self.result.x
        
        #Unit vectors
        self.normalHat, __ = faultUnitVectors(self.strike, self.dip, 0)
        self.strikeHat, self.dipHat =  vectorsPlane(self.normalHat, self.strike)
        
        #Normal components
        self.a, self.b, self.c = self.normalHat
        
        #Error Total
        self.error = self.result.fun
        
        # Distances from plane
        self.distEach = distFromPlane(
                self.xP, self.yP, self.zP,
                self.xC, self.yC, self.zC,
                self.a, self.b, self.c
                )

In [1]:
class interp(object):
    def __init__(self, x, y, z, 
                 strikeHyp=None, dipHyp=None, strike2Hyp=None, dip2Hyp=None, focalBool=None,
                 magnitudeWeight=None, exp=2, smooth=None,
                 splineNodesStrike = 7, splineNodesDip = 7, 
                 gridNodesStrike=50, gridNodesDip=50, eps=.1, 
                 minDist=None, minPlaneDist = None, cutPoints = None,
                 useSmoothSpline = False):
        self.x=x; self.y=y; self.z=z
        self.p = np.array( [self.x, self.y, self.z] ).T
        
        self.strikeHyp = strikeHyp
        self.dipHyp = dipHyp
        self.strike2Hyp = strike2Hyp
        self.dip2Hyp = dip2Hyp
        if focalBool is None:
            focalBool = np.ones(x.size, dtype=bool)
        self.focalAvailable = focalBool
        
        self.splineNodesStrike = splineNodesStrike
        self.splineNodesDip = splineNodesDip
        self.gridNodesStrike = gridNodesStrike
        self.gridNodesDip = gridNodesDip
        self.eps=eps
        self.minDist=minDist
        self.minPlaneDist = minPlaneDist
        self.exp=exp
        self.smooth=smooth
        self.cutPoints = cutPoints ### 02/15/2020
        
        if magnitudeWeight is not None:
            self.weight = magnitudeWeight
        else:
            self.weight = np.ones(x.shape)
        
        self.makePlane() 
        
        if self.minPlaneDist is not None:
            self.discludePoints()
        
        self.pS, self.pD, self.pN=  self.forwardRotation()
        self.makeKnots()
        self.findSpline()
        self.strikeDipPlane(minDist=self.minDist, cutPoints = cutPoints) # added cut points 02/15/2020
        
    def makePlane(self, strike=None, dip=None):
        planeInfo = optimumPlane(self.x, self.y, self.z, exp = self.exp)
        self.strikeHat = planeInfo.strikeHat
        self.dipHat = planeInfo.dipHat
        self.normalHat = planeInfo.normalHat 
        self.strike = planeInfo.strike
        self.dip = planeInfo.dip

        self.xC = planeInfo.xC
        self.yC = planeInfo.yC
        self.zC = planeInfo.zC
        
    def discludePoints(self):
        nx, ny, nz = self.normalHat
        self.planeDist = nx * self.x + ny * self.y + nz * self.z - nx * self.xC - ny * self.yC - nz * self.zC
        self.nearPlane = np.abs(self.planeDist)<self.minPlaneDist
        
        
    def forwardRotation(self):
        
        # First, store convert unit vesectors to new coordinate system.
        # This is needed to backrotate later. 
        axisOrig = np.array([
            [1, 0, 0],
            [0, 1, 0],
            [0, 0, 1]    ])
        axisOrigPrime = rotateUniVec(axisOrig, self.strikeHat, self.dipHat, self.normalHat)
        self.e1Prime, self.e2Prime, self.e3Prime = axisOrigPrime
        
        # Now, transform the points for the new coordinate system. 
        self.pPrime = rotateUniVec(self.p-np.array([ self.xC, self.yC, self.zC ]) , 
                                   self.strikeHat, self.dipHat, self.normalHat)
        return self.pPrime[:,0], self.pPrime[:,1], self.pPrime[:,2]
        
    def backRotation(self, pS, pD, pN, unCenter=True):
        # TODO: backRotation accepts any input points, but it always stores them in the interp variables.
        # If I need to backRotate different points within the one object, I need to store
        # the points with different names. 
        pInterpPrime = np.array([pS, pD, pN]).T
        pReturn = rotateUniVec(pInterpPrime, self.e1Prime, self.e2Prime, self.e3Prime)
        if unCenter:
            pReturn = pReturn + np.array([ self.xC, self.yC, self.zC ])
        return pReturn[:, 0], pReturn[:, 1], pReturn[:, 2]
  
    def makeKnots(self):
        self.knotS = np.linspace(np.amin(self.pS)+1, np.amax(self.pS)-1, 
                                 num = self.splineNodesStrike)
        self.knotD = np.linspace(np.amin(self.pD)+1, np.amax(self.pD)-1, 
                                 num = self.splineNodesDip)
    
#     if not self.useSmoothSpline:
    def findSpline(self):
        try:
            use = self.nearPlane
        except:
            use = np.ones(self.pS.size, dtype = bool)

        self.spline = scipy.interpolate.LSQBivariateSpline(self.pS[use], self.pD[use], self.pN[use],
            self.knotS, self.knotD, eps = self.eps, w = self.weight[use])#, s=self.smooth)
        
#     if self.useSmoothSpline:
#     def findSpline(self):
#         print('USING SMOOTH SPLINE NOW, smoothing = ', self.smooth)
#         try:
#             use = self.nearPlane
#         except:
#             use = np.ones(self.pS.size, dtype = bool)

#         self.spline = scipy.interpolate.SmoothBivariateSpline(self.pS[use], self.pD[use], self.pN[use],
#             eps = self.eps, s=self.smooth)#, w = self.weight[use], self.knotS, self.knotD, )
        
    def averageError(self):
        # Find average distance from surface to points
        nsp = self.spline.ev(self.pS, self.pD)
        errAv = np.average(np.abs(nsp - n))
        self.errAv = errAv
        return self.errAv
        
    def strikeDipPlane(self, cutPoints = True, minDist = None, sEdgeShift = 0, dEdgeShift = 0):
        """Note that this converts xyz coordinates into 2d strike and dip distances.
        It does not find angles!"""
        
        if minDist is not None:
            print('don\'t use minDist')
        
        gridS = np.linspace(np.amin(self.pS)+sEdgeShift, 
                            np.amax(self.pS)-sEdgeShift, self.gridNodesStrike)
        gridD = np.linspace(np.amin(self.pD)+dEdgeShift,
                            np.amax(self.pD)-dEdgeShift, self.gridNodesDip)
        
        # These are used in simpleCellWidths later, but I wish to remove this strategy
        self.gridS = gridS
        self.gridD = gridD
        
        self.interpD, self.interpS = np.meshgrid(gridD, gridS)
        self.interpS = self.interpS.ravel()
        self.interpD = self.interpD.ravel()
        
        if cutPoints:
            hull = ConvexHull( np.array([self.pS, self.pD]).T )
            verts = np.array([self.pS[hull.vertices], self.pD[hull.vertices]])
            path = mpltPath.Path( verts.T )
            keep = path.contains_points( np.array([self.interpS, self.interpD]).T )
            self.keep = keep
            
            if minDist is not None:
                keepDist = np.zeros( self.interpS.size , dtype = bool)
                for i in np.arange(self.interpS.size):
                    tempmin = np.amin( np.sqrt( (self.interpS[i]-self.pS)**2 +
                                                (self.interpD[i]-self.pD)**2 ) 
                                     )
                    if tempmin < minDist:
                        keepDist[i] = True
                    else:
                        keepDist[i] = False
                                      
                self.keep = np.logical_or( self.keep, keepDist )
                
                

            self.interpS = self.interpS[self.keep]
            self.interpD = self.interpD[self.keep]
        
#         self.backRotation(self.interpS, self.interpD, self.interpZ)
        
#     def runConvexHull(self, pS, pD):
#         hull = ConvexHull( np.array([pS, pD]).T )
#         verts = np.array([pS[hull.vertices], pD[hull.vertices]])
#         path = mpltPath.Path( verts.T )
#         keep = path.contains_points( np.array([self.interpS, self.interpD]).T )
        
    def simpleCellWidths(self):
        self.strikeWidth = (self.gridS[1]-self.gridS[0]) * np.ones(self.interpX.shape)
        self.dipWidth = (self.gridD[1]-self.gridD[0]) * np.ones(self.interpX.shape)
        #Should combine this with the convex hull function
        
    def planeInterp(self):
        planeN = np.zeros(self.interpS.size)
        self.planeX, self.planeY, self.planeZ = self.backRotation(self.interpS, self.interpD, planeN)
        
    def splineInterp(self, cutPoints=True, sEdgeShift = 0, dEdgeShift = 0):   
        self.strikeDipPlane(cutPoints=cutPoints, 
                            sEdgeShift = sEdgeShift, dEdgeShift = dEdgeShift)
        self.interpN = self.spline.ev(self.interpS, self.interpD)
        self.interpX, self.interpY, self.interpZ = self.backRotation(self.interpS, self.interpD, self.interpN)
        
    def surfaceNormal(self, ps=None, pd=None, dxMag = 1, dyMag = 1, backRotate = True):
        
        if ps is None or pd is None:
            ps = self.interpS
            pd = self.interpD
            
        dzdx = self.spline.ev(ps, pd, dx=1, dy=0)
        dzdy = self.spline.ev(ps, pd, dx=0, dy=1)
        
        tangentX = np.array([dxMag*np.ones(dzdx.shape), np.zeros(dzdx.shape), dzdx * dxMag]).T
        tangentY = np.array([np.zeros(dzdy.shape), dyMag*np.ones(dzdy.shape), dzdy * dyMag]).T
        
        cross = np.cross(tangentX, tangentY)
        
        for i in np.arange(cross[:,0].size):
            cross[i] = cross[i] / np.sqrt( cross[i,0]**2+cross[i,1]**2+cross[i,2]**2 )
            
        if backRotate:
            cross = self.backRotation(cross[:,0], cross[:,1], cross[:,2], unCenter=False)
        self.normalAprox = np.array([cross[0], cross[1], cross[2]]).T
        return self.normalAprox       
    
    def getJacobian(self, ps, pd):
        """
        ps and pd can be 1 or 2d
        jac is only evaluated at those points. no meshgrid is done. 
        """
        originalShape = ps.shape
        points = ps.size

        ones = np.ones(points)
        zeros = np.zeros(points)

        dnds = self.spline.ev(ps, pd, dx=1, dy=0).reshape(points)
        dndd = self.spline.ev(ps, pd, dx=0, dy=1).reshape(points)

        sTan = np.array([ones, zeros, dnds]).T
        dTan = np.array([zeros, ones, dndd]).T

        jac = zeros
        for i in np.arange(points):
            jac[i] = norm( np.cross(sTan[i], dTan[i]) )

        jac = jac.reshape(originalShape)

        return jac
        
    def compareFocal(self, ps=None, pd=None, altSurfaceNormal = None, minDistFromPlane=None,
                    maxDistFromPlane=None):
        try:
            boo = self.focalAvailable
        except:
            print('Get self.focalAvailable')

        if minDistFromPlane is not None:
            boo *= np.abs(self.pN) < minDistFromPlane
        if maxDistFromPlane is not None:
            boo *= np.abs(self.pN) > maxDistFromPlane



        if ps is None or pd is None:
            ps = self.pS[boo]
            pd = self.pD[boo]

        if altSurfaceNormal is None:
            nSurface = self.surfaceNormal(ps, pd)
        else:
            nSurface = altSurfaceNormal.reshape((1, 3)) * np.ones((self.dipHyp[boo].size,3))
        print(nSurface.shape)

        if self.strike2Hyp is None or self.dip2Hyp is None:
            print('condition')
            self.strike2Hyp=self.strikeHyp
            self.dip2Hyp = self.dipHyp

        n1, __ = faultUnitVectors(self.strikeHyp[boo], self.dipHyp[boo], 0 * self.strikeHyp[boo])
        n2, __ = faultUnitVectors(self.strike2Hyp[boo], self.dip2Hyp[boo], 0 * self.strike2Hyp[boo])

        self.theta1 = angleBetweenVectors(nSurface, n1)
        self.theta2 = angleBetweenVectors(nSurface, n2)


        zer = np.zeros(nSurface[:,0].size)

        self.strikeError = angleBetweenVectors(
            np.array([nSurface[:,0], nSurface[:,1], zer ]).T ,
            np.array([n1[:,0], n1[:,1], zer ]).T 
        )

        self.strikeError2 = angleBetweenVectors(
            np.array([nSurface[:,0], nSurface[:,1], zer ]).T ,
            np.array([n2[:,0]+1e-10, n2[:,1]+1e-10, zer ]).T 
        )
        self.n2Temp = n2

        self.dipError = angleBetweenVectors(
            np.array([zer, 
                      np.sqrt(nSurface[:,0]**2+nSurface[:,1]**2)
                      , nSurface[:,2] ]).T ,

            np.array([zer, 
                      np.sqrt(n1[:,0]**2+n1[:,1]**2), 
                      n1[:,2] ]).T
        )

        self.dipError2 = angleBetweenVectors(
            np.array([zer, 
                      np.sqrt(nSurface[:,0]**2+nSurface[:,1]**2), 
                      nSurface[:,2] ]).T ,

            np.array([zer, 
                      np.sqrt(n2[:,0]**2+n2[:,1]**2), 
                      n2[:,2] ]).T 
        )

        less1 = np.abs(self.theta1)<np.abs(self.theta2)

        self.thetaLeast = self.theta2.copy()
        self.thetaLeast[less1] = self.theta1[less1]
        
        
    def makeScatter(self, x, y, z):
        minVal = np.amin([self.interpN, self.pN])
        maxVal = np.amax([self.interpN, self.pN])
        plt.scatter(x, y, c = z, vmin = minVal, vmax = maxVal)
        plt.colorbar()
        plt.show()
        
    def plotInterpPlane(self):
        self.makeScatter(self.interpS, self.interpD, self.interpN)
    def plotDataPlane(self):
        self.makeScatter(self.pS, self.pD, self.pN)
        
    def saveVTK(self, data=None):
        if data is None:
            data = np.ones(self.interpX.size)
        
        writeVTKPoints(self.fName, self.interpX, self.interpY, self.interpZ,
                      data )
        
    def saveVTKPlane(self):
        writeVTKPoints(self.fName, self.planeX, self.planeY, self.planeZ,
                        np.zeros(self.interpX.size) )
        
    def saveVTKPoints(self, data=None, fName=None, onlyIfFocal=False):
        if fName is None:
            fName = self.fName
        if data is None:
            data = np.ones(self.x.size)
            
        if onlyIfFocal:
            boo = self.focalAvailable
        else:
            boo = np.ones(self.x.size, dtype = bool)
            
        writeVTKPoints(fName, self.x[boo], self.y[boo], self.z[boo], data=data[boo])

     # This is defined outside of class so that it is easy to update. 
    def figure3DSurface(self, otherx=None, othery=None, otherz=None, save = False, saveName = 'delete',
                         pointSize=.5, pLineWidthR=1/4):

        ###### scatter plot
        pointSize=pointSize
        scatterPlot = [go.Scatter3d(
            x=self.x,
            y=self.y,
            z=self.z,
            mode='markers',
            marker=dict(
                size=pointSize,
                color='green',
                line=dict(
                        color='rgb(0,0,0)',
                        width=pLineWidthR * pointSize
                    ),
                opacity=1) )]
        ######    

        if otherx is not None or othery is not None or otherz is not None:
            pointSize=pointSize
            scatterPlot2 = [go.Scatter3d(
                x=otherx,
                y=othery,
                z=otherz,
                mode='markers',
                marker=dict(
                    size=pointSize,
                    color='red',
                    line=dict(
                            color='rgb(0,0,0)',
                            width=pLineWidthR * pointSize
                        ),
                    opacity=1) )]
            scatterPlot = scatterPlot+scatterPlot2



        ###### Triangulation surface
        points2D=np.vstack([self.interpS,self.interpD]).T
        tri=Delaunay(points2D)

        surfacePlot=plotly_trisurf(self.interpX, self.interpY, self.interpZ,
                                   tri.simplices, colormap=cm.cubehelix, plot_edges=None)
        ######


        ###############
        ############################ 
        textFontDict = {'color':'red', 'size':18}
        ### north arrow
        xR = 10e3
        xNA = np.amax(xR-3e3)
        yNA = 28e3#np.amax(self.y)
        zNA = -14e3#np.amin(self.z)
        yBase = yNA - 5e3

        northLine = go.Scatter3d(
            x=[xNA, xNA], y=[yBase, yNA], z=[zNA, zNA],
            text = np.array(['', 'N']),
            mode='lines+text',#Change?
            line  =dict(width = 5, color = '#7f7f7f'),
            textposition = 'middle left',
            textfont = textFontDict
        )

        northCone = dict(
            type = 'cone',
            colorscale = 'Greys',
            showscale = False,
            x = [xNA], y = [yNA], z = [zNA],
            u = [0], v = [2e3], w = [0]
            )
        ###

        ###  scale bar      
        xN = np.array([xR, xR, xR])
        yN = np.array([-10e3, -5e3, 0])+yNA
        zN = np.array([zNA, zNA, zNA])
        scaleBar = go.Scatter3d(
            x=xN,
            y=yN,
            z=zN,
            text = np.array(['', '10 km', '']),
            mode='lines+text',
            line  =dict(width = 15, color = '#7f7f7f'),
            textposition='middle right',
            textfont = textFontDict
        )

        upLine = go.Scatter3d(
            x=[xNA, xNA], y=[yBase, yBase], z=[zNA, zNA+5e3],
            text = np.array(['', 'UP']),
            mode='lines+text',#Change?
            line  =dict(width = 5, color = '#7f7f7f'),
            textposition = 'middle left',
            textfont = textFontDict
        )

        upCone = dict(
            type = 'cone',
            colorscale = 'Greys',
            showscale = False,
            x = [xNA], y = [yBase], z = [zNA+5e3],
            u = [0], v = [0], w = [2e3]
            )
        ###
        surfacePlot.append(scaleBar)
        surfacePlot.append(northLine)
        surfacePlot.append(northCone)
        surfacePlot.append(upLine)
        surfacePlot.append(upCone)
        #########################
        ###############


#         layout = go.Layout(
#             autosize=True,
#             width=1000,
#             height=1000,
#             scene=dict(camera=dict(eye=dict(x=1.75, y=-0.7, z= 0.75) ),
#             xaxis = dict(title = 'East' ),
#             yaxis = dict(title = 'North'), 
#             zaxis = dict(title = 'Depth')  
#         ) )
        
        layout = go.Layout(
            showlegend=False,
            autosize=False,
            width=1000,
            height=1000,
            scene=dict(
                camera=dict(eye=dict(x=1.75, y=-0.7, z= 0.75) ),
                xaxis = dict(showticklabels=False, title=''),
                yaxis = dict(showticklabels=False, title=''),
                zaxis = dict(showticklabels=False, title='')
                      )  
            )

        fig2 = go.Figure(data=surfacePlot+scatterPlot, layout=layout)
        if save:
            py.iplot(fig2, filename = saveName)
        else:
            plot(fig2)